This wrapper is needed to enable use of the Fortran cuBLAS matmul program to run as a shared library in python. Once loaded, you can run tf32_matmul(a,b) and get 12Tflops vs 4Tflops.

In [3]:
import ctypes
import numpy as np
import cupy as cp

# Load the Fortran shared library
_lib = ctypes.CDLL('./fast_matmul.so')

# Define the function prototype
_lib.tf32_matmul.argtypes = [
    ctypes.c_void_p,  # A
    ctypes.c_void_p,  # B
    ctypes.c_void_p,  # C
    ctypes.c_int,     # M
    ctypes.c_int,     # N
    ctypes.c_int      # K
]

def tf32_matmul(a, b):
    """
    Fast matrix multiplication using cuBLAS TF32.
    """
    # Convert inputs to float32 if needed
    if isinstance(a, np.ndarray):
        a = cp.asarray(a, dtype=np.float32)
    if isinstance(b, np.ndarray):
        b = cp.asarray(b, dtype=np.float32)
    
    # Ensure float32
    a = cp.asarray(a, dtype=np.float32)
    b = cp.asarray(b, dtype=np.float32)
    
    # Get dimensions
    M, K = a.shape
    K2, N = b.shape
    assert K == K2, "Inner dimensions must match"
    
    # Create output array
    c = cp.empty((M, N), dtype=np.float32)
    
    # Call Fortran function
    _lib.tf32_matmul(
        ctypes.c_void_p(a.data.ptr),
        ctypes.c_void_p(b.data.ptr),
        ctypes.c_void_p(c.data.ptr),
        M, N, K
    )
    
    return c


In [4]:
if __name__ == "__main__":
    # Create test matrices
    a = np.random.rand(10000, 10000).astype(np.float32)
    b = np.random.rand(10000, 10000).astype(np.float32)
    
    # Using our fast matmul
    c_fast = tf32_matmul(a, b)
    
    # Compare with cupy
    a_cp = cp.asarray(a)
    b_cp = cp.asarray(b)
    c_cp = cp.matmul(a_cp, b_cp)
    
    # Check results
    print("Max difference:", cp.max(cp.abs(c_fast - c_cp)))
    
    # Benchmark
    import time
    
    # Warm up
    for _ in range(5):
        c_fast = tf32_matmul(a_cp, b_cp)
        c_cp = cp.matmul(a_cp, b_cp)
    cp.cuda.Stream.null.synchronize()
    
    # Time our implementation
    t0 = time.perf_counter()
    for _ in range(10):
        c_fast = tf32_matmul(a_cp, b_cp)
    cp.cuda.Stream.null.synchronize()
    t1 = time.perf_counter()
    print(f"Fast matmul: {(t1-t0)/10*1000:.2f} ms")
    
    # Time cupy
    t0 = time.perf_counter()
    for _ in range(10):
        c_cp = cp.matmul(a_cp, b_cp)
    cp.cuda.Stream.null.synchronize()
    t1 = time.perf_counter()
    print(f"CuPy matmul: {(t1-t0)/10*1000:.2f} ms")

Max difference: 168.52344
Fast matmul: 177.15 ms
CuPy matmul: 464.35 ms


cupy is much faster on RTX 4060 vs A1000 (only 2x difference vs 3-4x):
Max difference: 196.97314
Fast matmul: 146.03 ms
CuPy matmul: 264.85 ms

In [5]:
import numpy as np
import cupy as cp
from fast_matmul_wrapper import tf32_matmul
import time

# Print cupy configuration
print(f"cupy version: {cp.__version__}")
print(f"CUDA version: {cp.cuda.runtime.runtimeGetVersion()}")
print(f"Device: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")

# Try to get cuBLAS configuration
try:
    handle = cp.cuda.device.get_cublas_handle()
    print("cuBLAS handle obtained")
except:
    print("Could not get cuBLAS handle")

# Create test matrices; make sure not to normalise matricies or cupy will drop to 100Gflops!!
a = np.random.rand(5120, 5120).astype(np.float32)
b = np.random.rand(5120, 5120).astype(np.float32)

# Convert to cupy
a_cp = cp.asarray(a)
b_cp = cp.asarray(b)

# Function to run a single timed matmul
def timed_matmul(func, a, b, name=""):
    cp.cuda.Stream.null.synchronize()
    start = time.perf_counter()
    c = func(a, b)
    cp.cuda.Stream.null.synchronize()
    end = time.perf_counter()
    
    exec_time = end - start
    flops = 2.0 * float(5120**3)
    gflops = (flops / 1e9) / exec_time
    return gflops, c

# Try different ways of calling matmul
print("\nTesting different matmul approaches:")

# 1. Regular cupy matmul
gflops, c1 = timed_matmul(cp.matmul, a_cp, b_cp, "cupy.matmul")
print(f"cupy.matmul: {gflops:.2f} GFLOPS")

# 2. Our tf32_matmul
gflops, c2 = timed_matmul(tf32_matmul, a_cp, b_cp, "tf32_matmul")
print(f"tf32_matmul: {gflops:.2f} GFLOPS")

# Compare results
print(f"\nMax difference between implementations: {cp.max(cp.abs(c1 - c2))}")

# Now do full benchmark
def run_benchmark(func, name, warmup=5, runs=10):
    print(f"\nBenchmarking {name}...")
    
    # Warmup
    for _ in range(warmup):
        _ = func(a_cp, b_cp)
    cp.cuda.Stream.null.synchronize()
    
    # Timing runs
    times = []
    for i in range(runs):
        gflops, _ = timed_matmul(func, a_cp, b_cp)
        times.append(gflops)
        print(f"Run {i+1}: {gflops:.2f} GFLOPS")
    return times

# Run benchmarks
times_cupy = run_benchmark(cp.matmul, "cupy")
times_custom = run_benchmark(tf32_matmul, "tf32_matmul")

# Print statistics
def print_stats(name, times):
    print(f"\nPerformance Results for {name}:")
    print(f"  Minimum: {min(times):.2f} GFLOPS")
    print(f"  Maximum: {max(times):.2f} GFLOPS")
    print(f"  Average: {sum(times)/len(times):.2f} GFLOPS")
    print(f"  Std Dev: {np.std(times):.2f} GFLOPS")

print_stats("cupy", times_cupy)
print_stats("tf32_matmul", times_custom)


cupy version: 13.3.0
CUDA version: 12060
Device: NVIDIA RTX A1000 Laptop GPU
cuBLAS handle obtained

Testing different matmul approaches:
cupy.matmul: 3870.59 GFLOPS
tf32_matmul: 11330.21 GFLOPS

Max difference between implementations: 129.4381103515625

Benchmarking cupy...
Run 1: 3932.05 GFLOPS
Run 2: 3947.16 GFLOPS
Run 3: 3942.15 GFLOPS
Run 4: 3947.49 GFLOPS
Run 5: 3939.81 GFLOPS
Run 6: 3894.70 GFLOPS
Run 7: 3815.17 GFLOPS
Run 8: 3815.58 GFLOPS
Run 9: 3829.82 GFLOPS
Run 10: 3854.40 GFLOPS

Benchmarking tf32_matmul...
Run 1: 11743.99 GFLOPS
Run 2: 11760.96 GFLOPS
Run 3: 11752.08 GFLOPS
Run 4: 11757.11 GFLOPS
Run 5: 11750.89 GFLOPS
Run 6: 11743.22 GFLOPS
Run 7: 11746.86 GFLOPS
Run 8: 11712.84 GFLOPS
Run 9: 11713.53 GFLOPS
Run 10: 11762.01 GFLOPS

Performance Results for cupy:
  Minimum: 3815.17 GFLOPS
  Maximum: 3947.49 GFLOPS
  Average: 3891.83 GFLOPS
  Std Dev: 54.37 GFLOPS

Performance Results for tf32_matmul:
  Minimum: 11712.84 GFLOPS
  Maximum: 11762.01 GFLOPS
  Average: 11744.3

cupy version: 13.3.0
CUDA version: 12060
Device: NVIDIA GeForce RTX 4060
cuBLAS handle obtained

Testing different matmul approaches:
cupy.matmul: 8678.26 GFLOPS
tf32_matmul: 12975.45 GFLOPS

Max difference between implementations: 130.9873046875

Benchmarking cupy...
Run 1: 7368.19 GFLOPS
Run 2: 7264.97 GFLOPS
Run 3: 7343.90 GFLOPS
Run 4: 7289.49 GFLOPS
Run 5: 7249.27 GFLOPS
Run 6: 7283.76 GFLOPS
Run 7: 7248.43 GFLOPS
Run 8: 7171.27 GFLOPS
Run 9: 7401.01 GFLOPS
Run 10: 7240.99 GFLOPS

Benchmarking tf32_matmul...
Run 1: 13926.38 GFLOPS
Run 2: 13990.53 GFLOPS
Run 3: 13856.14 GFLOPS
Run 4: 14022.57 GFLOPS
Run 5: 13802.79 GFLOPS
Run 6: 13854.66 GFLOPS
Run 7: 13866.73 GFLOPS
Run 8: 13848.80 GFLOPS
Run 9: 13809.18 GFLOPS
Run 10: 13646.37 GFLOPS

Performance Results for cupy:
  Minimum: 7171.27 GFLOPS
  Maximum: 7401.01 GFLOPS
  Average: 7286.13 GFLOPS
  Std Dev: 64.61 GFLOPS

Performance Results for tf32_matmul:
  Minimum: 13646.37 GFLOPS
  Maximum: 14022.57 GFLOPS
  Average: 13862.41 GFLOPS
  Std Dev: 99.76 GFLOPS

In [6]:
# normalising values kills cupy performanc from 4,000gflops to only 100gflops!!
import numpy as np
import cupy as cp
import time

# Test both normal and normalized matrices
def test_both_versions():
    # Version 1: No normalization
    a1 = np.random.rand(5120, 5120).astype(np.float32)
    b1 = np.random.rand(5120, 5120).astype(np.float32)
    a1_cp = cp.asarray(a1)
    b1_cp = cp.asarray(b1)

    # Version 2: With normalization
    a2 = np.random.rand(5120, 5120).astype(np.float32)
    b2 = np.random.rand(5120, 5120).astype(np.float32)
    a2 = a2 / np.sqrt(a2.shape[1])
    b2 = b2 / np.sqrt(b2.shape[0])
    a2_cp = cp.asarray(a2)
    b2_cp = cp.asarray(b2)

    def benchmark(a, b, name):
        cp.cuda.Stream.null.synchronize()
        start = time.perf_counter()
        c = cp.matmul(a, b)
        cp.cuda.Stream.null.synchronize()
        end = time.perf_counter()
        
        exec_time = end - start
        flops = 2.0 * float(5120**3)
        gflops = (flops / 1e9) / exec_time
        return gflops, c

    print("Testing non-normalized matrices:")
    gflops1, c1 = benchmark(a1_cp, b1_cp, "non-normalized")
    print(f"GFLOPS: {gflops1:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c1))}")

    print("\nTesting normalized matrices:")
    gflops2, c2 = benchmark(a2_cp, b2_cp, "normalized")
    print(f"GFLOPS: {gflops2:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c2))}")

test_both_versions()


Testing non-normalized matrices:
GFLOPS: 4231.76
Max value in result: 1368.1602783203125

Testing normalized matrices:
GFLOPS: 105.38
Max value in result: 0.2668110103148127


The slowdown in the cuBLAS kernel is not as noticable (it is there but still running at 10Tflops).

In [7]:
import numpy as np
import cupy as cp
import time
from fast_matmul_wrapper import tf32_matmul

# Test both normal and normalized matrices
def test_both_versions():
    # Version 1: No normalization
    a1 = np.random.rand(5120, 5120).astype(np.float32)
    b1 = np.random.rand(5120, 5120).astype(np.float32)
    a1_cp = cp.asarray(a1)
    b1_cp = cp.asarray(b1)

    # Version 2: With normalization
    a2 = np.random.rand(5120, 5120).astype(np.float32)
    b2 = np.random.rand(5120, 5120).astype(np.float32)
    a2 = a2 / np.sqrt(a2.shape[1])
    b2 = b2 / np.sqrt(b2.shape[0])
    a2_cp = cp.asarray(a2)
    b2_cp = cp.asarray(b2)

    def benchmark_cupy(a, b):
        cp.cuda.Stream.null.synchronize()
        start = time.perf_counter()
        c = cp.matmul(a, b)
        cp.cuda.Stream.null.synchronize()
        end = time.perf_counter()
        
        exec_time = end - start
        flops = 2.0 * float(5120**3)
        gflops = (flops / 1e9) / exec_time
        return gflops, c

    def benchmark_tf32_matmul(a, b):
        cp.cuda.Stream.null.synchronize()
        start = time.perf_counter()
        c = tf32_matmul(a, b)
        cp.cuda.Stream.null.synchronize()
        end = time.perf_counter()
        
        exec_time = end - start
        flops = 2.0 * float(5120**3)
        gflops = (flops / 1e9) / exec_time
        return gflops, c

    # Test cupy.matmul
    print("Testing with cupy.matmul:")
    print("Non-normalized matrices:")
    gflops1, c1 = benchmark_cupy(a1_cp, b1_cp)
    print(f"GFLOPS: {gflops1:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c1))}")

    print("\nNormalized matrices:")
    gflops2, c2 = benchmark_cupy(a2_cp, b2_cp)
    print(f"GFLOPS: {gflops2:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c2))}")

    # Test tf32_matmul
    print("\nTesting with tf32_matmul:")
    print("Non-normalized matrices:")
    gflops3, c3 = benchmark_tf32_matmul(a1_cp, b1_cp)
    print(f"GFLOPS: {gflops3:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c3))}")

    print("\nNormalized matrices:")
    gflops4, c4 = benchmark_tf32_matmul(a2_cp, b2_cp)
    print(f"GFLOPS: {gflops4:.2f}")
    print(f"Max value in result: {cp.max(cp.abs(c4))}")

    # Verify results match between implementations
    print("\nVerifying results:")
    print("Non-normalized max difference:", cp.max(cp.abs(c1 - c3)))
    print("Normalized max difference:", cp.max(cp.abs(c2 - c4)))

test_both_versions()


Testing with cupy.matmul:
Non-normalized matrices:
GFLOPS: 4239.47
Max value in result: 1364.2012939453125

Normalized matrices:
GFLOPS: 106.19
Max value in result: 0.26554513465560065

Testing with tf32_matmul:
Non-normalized matrices:
GFLOPS: 14639.32
Max value in result: 1360.1336669921875

Normalized matrices:
GFLOPS: 10620.87
Max value in result: 0.2670082747936249

Verifying results:
Non-normalized max difference: 121.69861
Normalized max difference: 0.023368133077295183


RTX 4060 has much slower cuBLAS performance for normalised matrices and much higher cupy

Testing with cupy.matmul:
Non-normalized matrices:
GFLOPS: 8001.40
Max value in result: 1361.484619140625

Normalized matrices:
GFLOPS: 203.98
Max value in result: 0.26614665120703523

Testing with tf32_matmul:
Non-normalized matrices:
GFLOPS: 14230.39
Max value in result: 1362.4464111328125

Normalized matrices:
GFLOPS: 3094.93
Max value in result: 0.2662442624568939

Verifying results:
Non-normalized max difference: 122.25574
Normalized max difference: 0.02260998545646506

A1000 tf32_matmul suffers less of a slowdown with normalised:

Testing with cupy.matmul:
Non-normalized matrices:
GFLOPS: 4239.47
Max value in result: 1364.2012939453125

Normalized matrices:
GFLOPS: 106.19
Max value in result: 0.26554513465560065

Testing with tf32_matmul:
Non-normalized matrices:
GFLOPS: 14639.32
Max value in result: 1360.1336669921875

Normalized matrices:
GFLOPS: 10620.87
Max value in result: 0.2670082747936249

Verifying results:
Non-normalized max difference: 121.69861
Normalized max difference: 0.023368133077295183